In [4]:
import pandas as pd
import numpy as np
import ta  # Technical analysis library
import yfinance as yf
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pytz  # For timezone conversion
import warnings

warnings.filterwarnings('ignore')

# Define symbols and training data path
symbols = ['AUCTION', 'RAY', 'T', 'FLOW', 'ARKM', 'LAYER', 'POLYX', 'ETH', 'BTT', 
           '1INCH', 'RED', 'SOL', 'XRP', 'BTC', 'TRX']
train_path = "data/1-year-data/"  # Training data path


# Function to calculate technical indicators
def add_technical_indicators(df):
    df = df.copy()

    # Moving Averages
    df["SMA_10"] = ta.trend.sma_indicator(df["Close"], window=10)
    df["EMA_10"] = ta.trend.ema_indicator(df["Close"], window=10)

    # Volatility Indicators
    df["ATR"] = ta.volatility.average_true_range(df["High"], df["Low"], df["Close"], window=14)
    df["Bollinger_Upper"] = ta.volatility.bollinger_hband(df["Close"], window=20)
    df["Bollinger_Middle"] = ta.volatility.bollinger_mavg(df["Close"], window=20)
    df["Bollinger_Lower"] = ta.volatility.bollinger_lband(df["Close"], window=20)

    # Momentum Indicators
    df["RSI"] = ta.momentum.rsi(df["Close"], window=14)
    df["MACD"] = ta.trend.macd(df["Close"])

    # Volume Indicator
    df["OBV"] = ta.volume.on_balance_volume(df["Close"], df["Volume"])
    
    # Define target (price increase by 3% in the next hour)
    df['Target'] = np.where(df["Close"].shift(-1) >= df["Close"] * 1.02, 1, 0)

    # Drop NaN values generated from indicators
    df = df.dropna()

    return df


# Set Indian Standard Time (IST) timezone
ist = pytz.timezone("Asia/Kolkata")

# Store results
predicted_symbols = []

print("Processing...")

for symbol in symbols:
    try:
        # Read training data
        train_df = pd.read_csv(train_path + f"{symbol}-USD.csv")

        # Download test data from Yahoo Finance
        test_df = yf.download(f"{symbol}-USD", interval="1h", period="5d")

        # Check if we have enough data
        if len(test_df) < 20:
            print(f"Skipping {symbol} - Not enough data")
            continue

        # Convert timezone to IST
        test_df.index = test_df.index.tz_convert(ist)

        # Reset index and rename columns
        test_df.columns = [i[0] for i in test_df.columns]
        # Apply technical indicators
        train_df = add_technical_indicators(train_df)
        test_df = add_technical_indicators(test_df)

        # Remove last row from training data as it doesn't have a valid target
        train_df = train_df.iloc[:-1]

        # Prepare features and target
        X_train = train_df.drop(columns=["Target", "Datetime"])
        y_train = train_df["Target"]

        X_test = test_df.drop(columns=["Target"])
        y_test = test_df["Target"]

        # Train XGBoost Classifier
        model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        model.fit(X_train, y_train)

        # Make predictions for the next hour
        y_pred = model.predict(X_test)

        # If any prediction is 1 (indicating a price increase), store the symbol
        if 1 in y_pred:
            predicted_symbols.append(symbol)

    except Exception as e:
        print(f"Error processing {symbol}: {e}")

# to Print results
if predicted_symbols:
    print("\nSymbols predicted to increase by 3% in the next hour:")
    print(", ".join(predicted_symbols))
else:
    print("\nNo symbols predicted to increase by 3% in the next hour.")

[*********************100%***********************]  1 of 1 completed

Processing...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Symbols predicted to increase by 3% in the next hour:
AUCTION, ARKM


In [2]:
!pip install cctx